# Generate the JSON file for the Host of the First Repeater
    v1 -- Ball rolling

In [2]:
# imports
from importlib import reload

import importlib_resources

from astropy.coordinates import SkyCoord
from astropy import units

from frb.galaxies import frbgalaxy, defs

## The following will become a script that will be kept update (this Notebook may not)

## Instantiate

### Grab coordinates 

#### Marcote et al. 2017 for FRB

In [3]:
FRB_coord = SkyCoord('05h31m58.698s +33d8m52.59s', frame='icrs')
FRB_coord

<SkyCoord (ICRS): (ra, dec) in deg
    (82.994575, 33.14794167)>

#### Eyeball Tendulkar+17 offset

In [4]:
gal_coord = FRB_coord.directional_offset_by(-45*units.deg, 286e-3*units.arcsec)

In [5]:
gal_coord

<SkyCoord (ICRS): (ra, dec) in deg
    (82.99450791, 33.14799784)>

### Go

In [6]:
host121102 = frbgalaxy.FRBHost(gal_coord.ra.value, gal_coord.dec.value, '121102')
host121102

<FRBHost: 05:31:58.6819 +33:08:52.7922, FRB=121102>

## Redshift

In [7]:
host121102.set_z(0.19273, 'spec', err=0.00008)

In [8]:
host121102.redshift

{'z': 0.19273,
 'z_FRB': 0.19273,
 'z_FRB_err': 8e-05,
 'z_err': 8e-05,
 'z_spec': 0.19273,
 'z_spec_err': 8e-05}

In [9]:
host121102.z

0.19273

## Photometry

In [10]:
# None reported (directly)

## Nebular Lines

### Table 1 of Tengulkar+17 -- These include Galactic Extinction

In [24]:
neb_lines = {}
neb_lines['Ha'] = 0.652e-16
neb_lines['Ha_err'] = 0.009e-16
neb_lines['Ha_Al'] = 0.622
#
neb_lines['Hb'] = 0.118e-16
neb_lines['Hb_err'] = 0.011e-16
neb_lines['Hb_Al'] = 0.941
# 
neb_lines['[OIII] 5007'] = 0.575e-16
neb_lines['[OIII] 5007_err'] = 0.011e-16
neb_lines['[OIII] 5007_Al'] = 0.911
#
neb_lines['[NII] 6583'] = 0.030e-16 #* units.erg/units.cm**2/units.s      # Upper limit
neb_lines['[NII] 6583_err'] = -999. #* units.erg/units.cm**2/units.s
neb_lines['[NII] 6583_Al'] = 0.619 

In [25]:
AV = 2.42

In [29]:
host121102.neb_lines ={}
for key in neb_lines.keys():
    if '_err' in key:
        continue
    if 'Al' in key:
        continue
    # Ingest
    host121102.neb_lines[key] = neb_lines[key] * 10**(neb_lines[key+'_Al']*AV/2.5)
    if neb_lines[key+'_err'] > 0:
        host121102.neb_lines[key+'_err'] = neb_lines[key+'_err'] * 10**(neb_lines[key+'_Al']*AV/2.5)
    else:
        host121102.neb_lines[key+'_err'] = neb_lines[key+'_err']

In [30]:
host121102.neb_lines

{'Ha': 2.6082162461507154e-16,
 'Ha_err': 3.600298499287797e-18,
 'Hb': 9.611031647379592e-17,
 'Hb_err': 8.959436281455551e-18,
 '[NII] 6583': 1.192101555659964e-17,
 '[NII] 6583_err': -999.0,
 '[OIII] 5007': 4.3804209229532963e-16,
 '[OIII] 5007_err': 8.379935678693262e-18}

### Vette

In [31]:
for key in host121102.neb_lines.keys():
    if '_err' in key:
        continue
    assert key in defs.valid_neb_lines

## Morphology

### Values from Tendulkar+17

In [32]:
host121102.morphology['reff_ang'] = 0.41
host121102.morphology['reff_ang_err'] = 0.06
#
host121102.morphology['n'] = 2.2
host121102.morphology['n_err'] = 1.5
# 
host121102.morphology['b/a'] = 0.25
host121102.morphology['b/a_err'] = 0.13

In [33]:
for key in host121102.morphology.keys():
    if '_err' in key:
        continue
    assert key in defs.valid_morphology

## Derived

In [34]:
host121102.derived['M_r'] = -17.0  # AB; Tendulkar+17
host121102.derived['SFR_nebular'] = 0.23  # MSun/yr; Tendulkar+17
host121102.derived['Mstar'] = 5.5e7 # Msun; Tendulkar+17
host121102.derived['Mstar_err'] = 1.5e7 # Msun; Tendulkar+17

## Write

In [35]:
path = str(importlib_resources.files('frb.data.Galaxies.121102'))
path

'/home/xavier/Projects/FRB/frb/data/Galaxies/121102'

In [37]:
try:
    host121102.write_to_json(path=path, overwrite=False)
except IOError:
    pass